# Load occupancy data: RC-60-Mix

In [1]:
import numpy as np 
import pandas as pd

In [2]:
root_folder = 'preprocessed_data/'
f_x = root_folder + '058_5p_to_1p_X.txt'
f_y = root_folder + '058_5p_to_1p_y.txt'
data = pd.read_csv(f_x, delimiter=' ', header = None)
labels = pd.read_csv(f_y, delimiter=' ', header = None)
print(data.shape, labels.shape)

(4340, 5976) (4340, 1)


# Transfer exact occupancy labels to HOV/LOV 

In [4]:
def transfer_to_HOV_LOV(labels):# HOV: label=1 ; LOV: label=2
    if len(labels.shape)==2:
        labels = np.squeeze(labels).tolist()
    for idx, l in enumerate(labels):
        if l == 1:#5people: HOV
            labels[idx]=1
        elif l==2:#4people: HOV
            labels[idx]=1
        elif l==3:#3people
            labels[idx]=1
        elif l==4:#2people
            labels[idx]=2
        elif l==5: #1person
            labels[idx]=2
        else:
            raise SystemExit("invalid label")
    labels = np.array(labels)
    labels = np.expand_dims(labels, axis=1)
    return labels

In [8]:
new_labels = transfer_to_HOV_LOV(labels.copy())

In [12]:
new_labels = pd.DataFrame(new_labels)

# Train 1d cnn on RC-60-Mix

In [3]:
from das_lib.models import cnn1d
from das_lib.funs import split_train_test_2d
import tensorflow as tf

2023-07-10 14:25:56.093783: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [10]:
import random

def split_train_test_1d(data, labels, frac=.8, index=None):
    x_train = data.sample(frac=frac) if index is None else data.iloc[index]
    x_test = data.drop(x_train.index)
    y_train = labels.iloc[x_train.index]
    y_test = labels.drop(x_train.index)
    return x_train, x_test, y_train, y_test, x_train.index

In [14]:
x_train, x_test, y_train, y_test, x_train_index = split_train_test_1d(data, new_labels, frac=.67, index=None)
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
num_classes = 2
y_train = tf.keras.utils.to_categorical(y_train-1, num_classes)
y_test = tf.keras.utils.to_categorical(y_test-1, num_classes)

In [15]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2908, 5976, 1), (1432, 5976, 1), (2908, 2), (1432, 2))

In [16]:
m = cnn1d( (x_train.shape[1],1), y_train.shape[1])
print(m.summary())

2023-07-10 14:31:58.486037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 14:32:00.801019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38251 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


Model: "1dcnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5976, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2957, 16)          1040      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 369, 16)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 169, 32)           16416     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 21, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 3, 16)             

In [17]:
H = m.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test), verbose=False)
m.evaluate(x_test, y_test, batch_size=64, verbose=True)

2023-07-10 14:32:57.040461: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2023-07-10 14:33:03.229544: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


23/23 [==============================] - 0s 4ms/step - loss: 0.6881 - accuracy: 0.8959


[0.6881490349769592, 0.8959497213363647]

In [18]:
checkpoint_folder = 'training_results'
model_name = '2way_1d'
checkpoint_path = checkpoint_folder+'/'+'{0}'.format(model_name)
m.save(checkpoint_path)

INFO:tensorflow:Assets written to: training_results/2way_1d/assets


INFO:tensorflow:Assets written to: training_results/2way_1d/assets


In [ ]:
from das_lib.evaluation import save_training_history
f_name = 'performance/2way_1d.json'
save_training_history(f_name, H)

# Read the trained model

In [2]:
import tensorflow as tf
checkpoint_folder = 'training_results'
model_name = '2way_1d'
checkpoint_path = checkpoint_folder+'/'+'{0}'.format(model_name)
m = tf.keras.models.load_model(checkpoint_path)


2023-07-10 15:30:20.617349: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 15:30:44.545048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:30:47.158483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38251 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


# Validate the trained model on Ind. RC-60-1p

In [3]:
def expand_data(data, label, num_classes):
    data= np.expand_dims(data,-1)
    label = np.array([label]*len(data))
    label = tf.keras.utils.to_categorical(label-1, num_classes)
    return data, label

In [4]:
from utils.sampling import extract_car_data
root_folder = 'preprocessed_data/'
f_x = root_folder+'026_X.txt' 
f_y = root_folder+'026_y.txt'
car_label = 2 # in speed data (023-027), car 2 is label = 2
car2_data =  extract_car_data(f_x, f_y, car_label)

In [5]:
label = 2 #new label, car2+1p->LOV
num_classes = 2 #HOV, LOV
car2_1p_data, car2_1p_label_new = expand_data(car2_data, label, num_classes)
print(car2_1p_data.shape, car2_1p_label_new.shape)
m.evaluate(car2_1p_data, car2_1p_label_new)

(451, 5976, 1) (451, 2)


2023-07-10 15:30:53.435703: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


15/15 [==============================] - 8s 5ms/step - loss: 7.0573 - accuracy: 0.4612


2023-07-10 15:30:58.767010: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[7.0573320388793945, 0.4611973464488983]

In [6]:
from das_lib.evaluation import save_wrong_indices
f_name = 'performance/wrong_indices/2way_1d_1p.json'
save_wrong_indices(f_name, m, car2_1p_data, car2_1p_label_new)

15/15 [==============================] - 0s 3ms/step


# Validate the trained model on Ind. RC-60-5p

In [7]:
root_folder = 'preprocessed_data/'
f_x = root_folder+'058_5p_X.txt' 
f_y = root_folder+'058_5p_y.txt'
car_label = 1 #old label, direction west => label=1
car2_data = extract_car_data(f_x, f_y, car_label)

In [8]:
label = 1 #new label, 5p in car 2 = > label = 1 
num_classes = 2
car2_5p_data, car2_5p_label_new = expand_data(car2_data, label, num_classes)
print(car2_5p_data.shape, car2_5p_label_new.shape)

(291, 5976, 1) (291, 2)


In [9]:
m.evaluate(car2_5p_data, car2_5p_label_new)

10/10 [==============================] - 0s 3ms/step - loss: 14.2821 - accuracy: 0.2474


[14.282137870788574, 0.24742268025875092]

In [10]:
f_name = 'performance/wrong_indices/2way_1d_5p.json'
save_wrong_indices(f_name, m, car2_5p_data, car2_5p_label_new)

10/10 [==============================] - 0s 2ms/step
